# Predicting Repeat Buyers

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
%matplotlib inline

C:\Users\Jared Chung\Anaconda2\lib\site-packages\matplotlib\__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [3]:
train = pd.read_csv("trainHistory.csv", parse_dates =['offerdate'])
test= pd.read_csv("testHistory.csv",parse_dates =['offerdate'])
offer = pd.read_csv("offers.csv")

In [5]:
# Data Reduction Script used from Triskelion 
transactions = pd.read_csv("../transactions/reduced2.csv",parse_dates =['date'])

## Preprocessing Data

In [31]:
merged = pd.merge(train,offer,left_on='offer',right_on='offer',how='left')

In [35]:
def preprocessing(x):
    
    x['year'] = x.offerdate.apply(lambda n: n.year)
    x['month'] = x.offerdate.apply(lambda n: n.month)
    x['week'] = x.offerdate.apply(lambda n: n.week)
    x['dayofweek'] = x.offerdate.apply(lambda n: n.dayofweek)
    return x

In [36]:
merged = preprocessing(merged)

In [34]:
merged.head()

,id,chain,offer,market,repeattrips,repeater,offerdate,category,quantity,company,offervalue,brand
0,86246,205,1208251,34,5,t,2013-04-24,2202,1,104460040,2.00,3718
1,86252,205,1197502,34,16,t,2013-03-27,3203,1,106414464,0.75,13474
2,12682470,18,1197502,11,0,f,2013-03-28,3203,1,106414464,0.75,13474
3,12996040,15,1197502,9,0,f,2013-03-25,3203,1,106414464,0.75,13474
4,13089312,15,1204821,9,0,f,2013-04-01,5619,1,107717272,1.50,102504


In [8]:
transactions.head()

,id,chain,dept,category,company,brand,date,productsize,productmeasure,purchasequantity,purchaseamount
0,86246,205,99,9909,104538848,15343,2012-03-02,16,OZ,1,2.49
1,86246,205,21,2106,105100050,27873,2012-03-02,64,OZ,1,3.29
2,86246,205,26,2630,103700030,14647,2012-03-02,56,CT,1,1.00
3,86246,205,58,5824,108674585,55172,2012-03-02,16,OZ,1,3.29
4,86246,205,26,2634,103700030,3293,2012-03-06,6,RL,1,4.99


In [26]:
x = transactions.groupby(['id','category']).size()
x = pd.DataFrame(x).reset_index()
x.columns = ['id', 'category', 'category'+'_quantity_bought']
x

,id,category,category_quantity_bought
0,86246,305,3
1,86246,610,1
2,86246,703,1
3,86246,706,9
4,86246,708,3
5,86246,803,1
6,86246,809,8
7,86246,818,24
8,86246,1703,15
9,86246,1704,2


In [28]:
pd.merge?

In [33]:
z = pd.merge(merged,x,on=['id','category'],how='left')
z.head()

,id,chain,offer,market,repeattrips,repeater,offerdate,category,quantity,company,offervalue,brand,category_quantity_bought
0,86246,205,1208251,34,5,t,2013-04-24,2202,1,104460040,2.00,3718,NaN
1,86252,205,1197502,34,16,t,2013-03-27,3203,1,106414464,0.75,13474,6
2,12682470,18,1197502,11,0,f,2013-03-28,3203,1,106414464,0.75,13474,1
3,12996040,15,1197502,9,0,f,2013-03-25,3203,1,106414464,0.75,13474,NaN
4,13089312,15,1204821,9,0,f,2013-04-01,5619,1,107717272,1.50,102504,NaN


In [7]:
transactions['year'] = transactions.date.apply(lambda n: n.year)
transactions['month'] = transactions.date.apply(lambda n: n.month)
transactions['week'] = transactions.date.apply(lambda n: n.week)
transactions['dayofweek'] = transactions.date.apply(lambda n: n.dayofweek)

In [16]:
len(train.chain.unique())

130

In [ ]:
# Mode l